In [2]:

# Installer les packages nécessaires
!pip install tensorflow tensorflow-hub librosa soundfile

import os
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import tensorflow as tf
import tensorflow_hub as hub
from pathlib import Path
from tqdm import tqdm
import pickle
from datetime import datetime

print("✅ Imports OK")
print(f"TensorFlow version: {tf.__version__}")

c:\Users\Rahmaa\Desktop\deep learning\.venv\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version



✅ Imports OK
TensorFlow version: 2.20.0


In [ ]:
# CELL 2: Configuration

# CONFIGURATION - Chemins mis à jour
AUDIO_FOLDER = r"C:\Users\Rahmaa\Desktop\deep learning\downloadSongs\audio_dataset_1000"
OUTPUT_DIR = r"C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database"

# Créer le dossier des embeddings
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*70)
print("🎵 EXTRACTION EMBEDDINGS YAMNET - 1000 CHANSONS")
print("="*70)
print(f"\n📁 Dossier audio: {AUDIO_FOLDER}")
print(f"💾 Dossier embeddings: {OUTPUT_DIR}")
print(f"⏱️ Durée estimée: ~8-10 heures")


🎵 EXTRACTION EMBEDDINGS YAMNET - 1000 CHANSONS

📁 Dossier audio: C:\Users\Rahmaa\Desktop\deep learning\downloadSongs\audio_dataset_1000
💾 Dossier embeddings: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database
⏱️ Durée estimée: ~8-10 heures


In [ ]:
# CELL 3: Lister les Fichiers Audio

print("\n" + "="*70)
print("📂 SCAN DES FICHIERS AUDIO")
print("="*70 + "\n")

# Extensions supportées
audio_extensions = ['.mp3', '.wav', '.m4a', '.flac', '.ogg', '.webm', '.opus']

# Trouver tous les fichiers audio
audio_files = []
for ext in audio_extensions:
    audio_files.extend(list(Path(AUDIO_FOLDER).glob(f'*{ext}')))

print(f"✅ Trouvé {len(audio_files)} fichiers audio\n")

if len(audio_files) == 0:
    print("❌ AUCUN FICHIER TROUVÉ!")
    print(f"Place tes fichiers dans: {AUDIO_FOLDER}")
    raise FileNotFoundError("Aucun fichier audio trouvé!")

# Afficher quelques exemples
print("📋 Exemples de fichiers:")
for i, file in enumerate(audio_files[:10], 1):
    print(f"   {i}. {file.name}")

if len(audio_files) > 10:
    print(f"   ... et {len(audio_files) - 10} autres fichiers")

print(f"\n✅ Dataset: {len(audio_files)} chansons prêtes")


📂 SCAN DES FICHIERS AUDIO

✅ Trouvé 1000 fichiers audio

📋 Exemples de fichiers:
   1. Adele - Love In The Dark.m4a
   2. Adele - Set Fire to the Rain.m4a
   3. Adele - When We Were Young.m4a
   4. aespa - aenergy.m4a
   5. aespa - Dirty Work.m4a
   6. aespa - Rich Man.m4a
   7. aespa - UP (KARINA Solo).m4a
   8. aespa - 자각몽 Lucid Dream.m4a
   9. Angham - Aktablk Tahod.m4a
   10. Angham - Ana Baatoh Kteer.m4a
   ... et 990 autres fichiers

✅ Dataset: 1000 chansons prêtes


In [ ]:
# CELL 4: Extraction des Métadonnées

print("\n" + "="*70)
print("📝 EXTRACTION DES MÉTADONNÉES")
print("="*70 + "\n")

# Créer un DataFrame avec les infos
songs_data = []

for audio_file in audio_files:
    filename = audio_file.stem 
    
    # Parser "Artist - Song" 
    if ' - ' in filename:
        parts = filename.split(' - ', 1)
        artist = parts[0].strip()
        song = parts[1].strip()
    else:
        artist = "Unknown"
        song = filename
    
    songs_data.append({
        'id': len(songs_data),
        'filename': audio_file.name,
        'filepath': str(audio_file),
        'artist': artist,
        'song': song
    })

df_songs = pd.DataFrame(songs_data)

print(f"✅ {len(df_songs)} chansons indexées\n")
print("📊 Aperçu des métadonnées:")
print(df_songs[['id', 'artist', 'song']].head(10))

print(f"\n📈 Statistiques:")
print(f"   • Total chansons: {len(df_songs)}")
print(f"   • Artistes uniques: {df_songs['artist'].nunique()}")




📝 EXTRACTION DES MÉTADONNÉES

✅ 1000 chansons indexées

📊 Aperçu des métadonnées:
   id  artist                  song
0   0   Adele      Love In The Dark
1   1   Adele  Set Fire to the Rain
2   2   Adele    When We Were Young
3   3   aespa               aenergy
4   4   aespa            Dirty Work
5   5   aespa              Rich Man
6   6   aespa      UP (KARINA Solo)
7   7   aespa       자각몽 Lucid Dream
8   8  Angham         Aktablk Tahod
9   9  Angham      Ana Baatoh Kteer

📈 Statistiques:
   • Total chansons: 1000
   • Artistes uniques: 52


In [ ]:
# CELL 5: Charger le Modèle YAMNet

print("\n" + "="*70)
print("🧠 CHARGEMENT DU MODÈLE YAMNET")
print("="*70 + "\n")

print("📥 Téléchargement de YAMNet depuis TensorFlow Hub...")
print("(Première fois: ~1-2 minutes)\n")

# Charger YAMNet pré-entraîné
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')

print("✅ YAMNet chargé avec succès!")
print("\n📊 Info sur YAMNet:")
print("   • Architecture: CNN (MobileNet-like)")
print("   • Pré-entraîné sur: AudioSet (2M+ clips)")
print("   • Input: Audio 16kHz mono")
print("   • Output: Embeddings 1024D")



🧠 CHARGEMENT DU MODÈLE YAMNET

📥 Téléchargement de YAMNet depuis TensorFlow Hub...
(Première fois: ~1-2 minutes)




✅ YAMNet chargé avec succès!

📊 Info sur YAMNet:
   • Architecture: CNN (MobileNet-like)
   • Pré-entraîné sur: AudioSet (2M+ clips)
   • Input: Audio 16kHz mono
   • Output: Embeddings 1024D


In [ ]:
# CELL 6: Fonction d'Extraction d'Embeddings

def extract_yamnet_embedding(audio_path, duration=30):
    """
    Extrait l'embedding YAMNet d'un fichier audio
    
    Args:
        audio_path: Chemin vers le fichier audio
        duration: Durée en secondes à analyser (30s = signature suffisante)
    
    Returns:
        embedding: Vector 1024D représentant l'audio
        error_msg: Message d'erreur si échec
    """
    try:
        waveform = None
        error_msg = None
        
        # Méthode 1: librosa normal
        try:
            waveform, sr = librosa.load(audio_path, sr=16000, duration=duration, mono=True)
        except Exception as e1:
            # Méthode 2: audioread
            try:
                import audioread
                with audioread.audio_open(audio_path) as f:
                    sr = f.samplerate
                    duration_frames = int(duration * sr) if duration else None
                    waveform = np.concatenate([np.frombuffer(buf, dtype=np.int16) 
                                              for buf in f])
                    if duration_frames:
                        waveform = waveform[:duration_frames]
                    waveform = waveform.astype(np.float32) / 32768.0
                    if sr != 16000:
                        waveform = librosa.resample(waveform, orig_sr=sr, target_sr=16000)
            except Exception as e2:
                error_msg = f"Échec chargement: {Path(audio_path).name}"
                return None, error_msg
        
        if waveform is None or len(waveform) == 0:
            return None, "Waveform vide"
        
        # Normaliser l'amplitude
        if np.abs(waveform).max() > 0:
            waveform = waveform / np.abs(waveform).max()
        
        # Extraire les embeddings avec YAMNet
        scores, embeddings, spectrogram = yamnet_model(waveform)
        
        # Moyenner les embeddings sur toute la durée
        embedding = np.mean(embeddings.numpy(), axis=0)
        
        return embedding, None
    
    except Exception as e:
        error_msg = f"{Path(audio_path).name}: {str(e)[:50]}"
        return None, error_msg


print("✅ Fonction d'extraction définie")


✅ Fonction d'extraction définie


In [ ]:
# CELL 7: Extraction des Embeddings - BATCH PROCESSING

print("\n" + "="*70)
print("🎵 EXTRACTION DES EMBEDDINGS - 1000 CHANSONS")
print("="*70 + "\n")

print(f"⏱️ Durée estimée: ~8-10 heures")
print(f"💡 Le script sauvegarde automatiquement tous les 100 fichiers\n")

# Initialisation
embeddings_list = []
failed_files = []
error_details = []
start_time = datetime.now()

# Traiter par batch avec sauvegarde régulière
BATCH_SIZE = 100

for idx, row in tqdm(df_songs.iterrows(), total=len(df_songs), desc="🎵 Extraction"):
    # Extraire embedding
    embedding, error_msg = extract_yamnet_embedding(row['filepath'], duration=30)
    
    if embedding is not None:
        embeddings_list.append(embedding)
    else:
        embeddings_list.append(np.zeros(1024))  # Embedding vide si échec
        failed_files.append(row['filename'])
        if error_msg:
            error_details.append(error_msg)
    
    # Sauvegarde intermédiaire tous les 100 fichiers
    if (idx + 1) % BATCH_SIZE == 0:
        elapsed = (datetime.now() - start_time).total_seconds() / 60
        remaining = ((len(df_songs) - idx - 1) / (idx + 1)) * elapsed
        
        print(f"\n   ✅ {idx + 1}/{len(df_songs)} traités")
        print(f"   ✅ Succès: {(idx + 1) - len(failed_files)}")
        print(f"   ❌ Échecs: {len(failed_files)}")
        print(f"   ⏱️ Temps écoulé: {elapsed:.1f} min")
        print(f"   ⏰ Temps restant estimé: {remaining:.1f} min\n")
        
        # Sauvegarde intermédiaire
        temp_embeddings = np.array(embeddings_list)
        temp_path = os.path.join(OUTPUT_DIR, f"embeddings_temp_{idx+1}.npy")
        np.save(temp_path, temp_embeddings)
        print(f"   💾 Sauvegarde temp: {temp_path}\n")

# Convertir en array final
embeddings_array = np.array(embeddings_list)

# Temps total
total_time = (datetime.now() - start_time).total_seconds() / 60

print("\n" + "="*70)
print("✅ EXTRACTION TERMINÉE!")
print("="*70)
print(f"📊 Shape des embeddings: {embeddings_array.shape}")
print(f"✅ Succès: {len(df_songs) - len(failed_files)}/{len(df_songs)}")
print(f"⏱️ Temps total: {total_time:.1f} minutes")

if failed_files:
    print(f"\n⚠️ {len(failed_files)} fichiers échoués:")
    for f in failed_files[:10]:
        print(f"   • {f}")
    if len(failed_files) > 10:
        print(f"   ... et {len(failed_files) - 10} autres")




🎵 EXTRACTION DES EMBEDDINGS - 1000 CHANSONS

⏱️ Durée estimée: ~8-10 heures
💡 Le script sauvegarde automatiquement tous les 100 fichiers



🎵 Extraction:   0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\Rahmaa\AppData\Local\Temp\ipykernel_7592\1773875211.py:24: UserWarning: PySoundFile failed. Trying audioread instead.
  waveform, sr = librosa.load(audio_path, sr=16000, duration=duration, mono=True)
c:\Users\Rahmaa\Desktop\deep learning\.venv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
🎵 Extraction:   0%|          | 1/1000 [00:04<1:15:51,  4.56s/it]C:\Users\Rahmaa\AppData\Local\Temp\ipykernel_7592\1773875211.py:24: UserWarning: PySoundFile failed. Trying audioread instead.
  waveform, sr = librosa.load(audio_path, sr=16000, duration=duration, mono=True)
c:\Users\Rahmaa\Desktop\deep learning\.venv\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version


   ✅ 100/1000 traités
   ✅ Succès: 100
   ❌ Échecs: 0
   ⏱️ Temps écoulé: 0.4 min
   ⏰ Temps restant estimé: 4.0 min

   💾 Sauvegarde temp: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database\embeddings_temp_100.npy



🎵 Extraction:  20%|██        | 200/1000 [00:47<02:54,  4.58it/s]


   ✅ 200/1000 traités
   ✅ Succès: 200
   ❌ Échecs: 0
   ⏱️ Temps écoulé: 0.8 min
   ⏰ Temps restant estimé: 3.2 min

   💾 Sauvegarde temp: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database\embeddings_temp_200.npy



...


   ✅ 800/1000 traités
   ✅ Succès: 800
   ❌ Échecs: 0
   ⏱️ Temps écoulé: 3.0 min
   ⏰ Temps restant estimé: 0.8 min

   💾 Sauvegarde temp: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database\embeddings_temp_800.npy



🎵 Extraction:  90%|█████████ | 901/1000 [03:22<00:19,  5.03it/s]


   ✅ 900/1000 traités
   ✅ Succès: 900
   ❌ Échecs: 0
   ⏱️ Temps écoulé: 3.4 min
   ⏰ Temps restant estimé: 0.4 min

   💾 Sauvegarde temp: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database\embeddings_temp_900.npy



🎵 Extraction: 100%|██████████| 1000/1000 [03:46<00:00,  4.42it/s]


   ✅ 1000/1000 traités
   ✅ Succès: 1000
   ❌ Échecs: 0
   ⏱️ Temps écoulé: 3.8 min
   ⏰ Temps restant estimé: 0.0 min

   💾 Sauvegarde temp: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database\embeddings_temp_1000.npy


✅ EXTRACTION TERMINÉE!
📊 Shape des embeddings: (1000, 1024)
✅ Succès: 1000/1000
⏱️ Temps total: 3.8 minutes


In [11]:
# ============================================================================
# CELL 8: Sauvegarde Finale de la Database
# ============================================================================

print("\n" + "="*70)
print("💾 SAUVEGARDE DE LA DATABASE")
print("="*70 + "\n")

# 1. Sauvegarder les embeddings
embeddings_path = os.path.join(OUTPUT_DIR, "audio_embeddings_yamnet_1000.npy")
np.save(embeddings_path, embeddings_array)
print(f"✅ Embeddings: {embeddings_path}")
print(f"   Shape: {embeddings_array.shape}")
print(f"   Taille: {embeddings_array.nbytes / (1024**2):.2f} MB")

# 2. Sauvegarder les métadonnées
metadata_path = os.path.join(OUTPUT_DIR, "songs_metadata_1000.csv")
df_songs.to_csv(metadata_path, index=False, encoding='utf-8-sig')
print(f"\n✅ Métadonnées: {metadata_path}")
print(f"   {len(df_songs)} chansons")

# 3. Sauvegarder la configuration
config = {
    'n_songs': len(df_songs),
    'embedding_dim': 1024,
    'model': 'YAMNet',
    'duration_seconds': 30,
    'sample_rate': 16000,
    'creation_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'failed_files': failed_files,
    'processing_time_minutes': total_time
}

config_path = os.path.join(OUTPUT_DIR, "database_config.pkl")
with open(config_path, 'wb') as f:
    pickle.dump(config, f)
print(f"\n✅ Configuration: {config_path}")

# 4. Sauvegarder rapport texte
report_path = os.path.join(OUTPUT_DIR, "extraction_report.txt")
with open(report_path, 'w', encoding='utf-8') as f:
    f.write("="*70 + "\n")
    f.write("RAPPORT D'EXTRACTION EMBEDDINGS YAMNET\n")
    f.write("="*70 + "\n\n")
    f.write(f"Date: {config['creation_date']}\n")
    f.write(f"Total chansons: {len(df_songs)}\n")
    f.write(f"Succès: {len(df_songs) - len(failed_files)}\n")
    f.write(f"Échecs: {len(failed_files)}\n")
    f.write(f"Temps total: {total_time:.1f} minutes\n\n")
    
    if failed_files:
        f.write("Fichiers échoués:\n")
        for file in failed_files:
            f.write(f"  - {file}\n")

print(f"✅ Rapport: {report_path}")

# Résumé final
print("\n" + "="*70)
print("🎉 DATABASE CRÉÉE AVEC SUCCÈS!")
print("="*70)
print(f"\n📁 Dossier: {OUTPUT_DIR}")
print(f"\n📦 Fichiers créés:")
print(f"   1. audio_embeddings_yamnet_1000.npy ({embeddings_array.shape})")
print(f"   2. songs_metadata_1000.csv ({len(df_songs)} chansons)")
print(f"   3. database_config.pkl")
print(f"   4. extraction_report.txt")
print("\n🎯 Prochaine étape:")
print("   → Utiliser ces embeddings pour reconnaissance en temps réel")
print("   → Comparer avec audio enregistré via cosine similarity")



💾 SAUVEGARDE DE LA DATABASE

✅ Embeddings: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database\audio_embeddings_yamnet_1000.npy
   Shape: (1000, 1024)
   Taille: 3.91 MB

✅ Métadonnées: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database\songs_metadata_1000.csv
   1000 chansons

✅ Configuration: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database\database_config.pkl
✅ Rapport: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database\extraction_report.txt

🎉 DATABASE CRÉÉE AVEC SUCCÈS!

📁 Dossier: C:\Users\Rahmaa\Desktop\deep learning\audio_embeddings_database

📦 Fichiers créés:
   1. audio_embeddings_yamnet_1000.npy ((1000, 1024))
   2. songs_metadata_1000.csv (1000 chansons)
   3. database_config.pkl
   4. extraction_report.txt

🎯 Prochaine étape:
   → Utiliser ces embeddings pour reconnaissance en temps réel
   → Comparer avec audio enregistré via cosine similarity


In [12]:
# CELL 9: Statistiques et Validation
# ============================================================================

print("\n" + "="*70)
print("📊 VALIDATION DE LA DATABASE")
print("="*70 + "\n")

# Vérifier qu'il n'y a pas d'embeddings nuls
zero_embeddings = np.sum(np.all(embeddings_array == 0, axis=1))
print(f"Embeddings valides: {len(embeddings_array) - zero_embeddings}/{len(embeddings_array)}")

# Statistiques sur les embeddings
print(f"\n📈 Statistiques des embeddings:")
print(f"   • Min: {embeddings_array.min():.4f}")
print(f"   • Max: {embeddings_array.max():.4f}")
print(f"   • Mean: {embeddings_array.mean():.4f}")
print(f"   • Std: {embeddings_array.std():.4f}")

# Top artistes
print(f"\n🎤 Top 10 Artistes:")
top_artists = df_songs['artist'].value_counts().head(10)
for artist, count in top_artists.items():
    print(f"   {count:3d}x {artist}")

print("\n✅ Validation terminée!")
print("\n🎵 Ta database de 1000 chansons est prête pour Shazam-like!")


📊 VALIDATION DE LA DATABASE

Embeddings valides: 1000/1000

📈 Statistiques des embeddings:
   • Min: 0.0000
   • Max: 7.0994
   • Mean: 0.0732
   • Std: 0.1620

🎤 Top 10 Artistes:
   877x NA
     9x Angham
     8x SEVENTEEN
     6x PNL
     6x TWICE
     6x Red Velvet
     5x Arctic Monkeys
     5x TOMORROW X TOGETHER
     5x aespa
     4x Haifa Wehbe

✅ Validation terminée!

🎵 Ta database de 1000 chansons est prête pour Shazam-like!
